# Code Written by:
**Shweta Tiwari**
*20 Oct 2023*

## Algorithm:  Counting Ones

In [1]:
import time

In [2]:
import numpy as np
from collections import defaultdict
from itertools import count

# Algorithm

In [3]:
%%time
def stream_counter():
    bucket = defaultdict(list)
    timestamp = count(1)
    estimate = None

    while True:
        code = yield estimate
        estimate = None

        # update buckets
        if code is True:
            bucket[1].append(next(timestamp))

            i = 1
            while len(bucket[i]) == 3:
                bucket[2 * i].append(bucket[i][1])
                del bucket[i][:2]
                i *= 2

        elif code is False:
            next(timestamp)

        # estimate count
        elif isinstance(code, int):
            counts = [i for i in bucket for t in bucket[i] if code < t] or [0]
            estimate = sum(counts) - counts[-1] // 2

        # debug
        elif code == 'debug':
            for i in bucket:
                print(i, bucket[i])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


# Run

In [4]:
%%time
n = 10 ** 6
ctr = stream_counter()
next(ctr)
for i in range(n):
    ctr.send(np.random.rand() >= .5)

for i in np.linspace(.99, 0, 5):
    k = int(i * n)
    print(f'last {n - k} bits: {ctr.send(k)}')

last 10000 bits: 5640
last 257501 bits: 137736
last 505000 bits: 301576
last 752500 bits: 432648
last 1000000 bits: 432648
CPU times: user 747 ms, sys: 6.12 ms, total: 753 ms
Wall time: 758 ms


In [5]:
%%time
ctr.send('debug')

1 [999998, 1000000]
2 [999997]
4 [999995]
8 [999979, 999990]
16 [999963]
32 [999941]
64 [999881]
128 [999764]
256 [999523]
512 [997989, 999023]
1024 [996953]
2048 [990879, 994987]
4096 [978563, 986884]
8192 [970369]
16384 [953971]
32768 [855386, 921089]
65536 [658613, 789935]
131072 [263593, 526704]
CPU times: user 210 µs, sys: 29 µs, total: 239 µs
Wall time: 241 µs


# The End